# Requirements

In [ ]:
! pip install transformers

In [ ]:
! pip install peft

In [ ]:
! pip install sentencepiece

In [ ]:
! pip install accelerate

In [ ]:
! pip install bitsandbytes

In [ ]:
! pip install --upgrade accelerate bitsandbytes

# Model

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig, GenerationConfig
from peft import PeftModel
import pandas as pd
import json
import time

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)

In [ ]:
alpaca_model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b", is_trainable=True)

In [ ]:
alpaca_model.print_trainable_parameters()

# Upload data

In [ ]:
df_path = 'path'

dataset = pd.read_csv(df_path, sep=";")

dataset.head()

In [ ]:
df = dataset[dataset['label'] == "(ce-ec)"]

In [ ]:
patterns = []

for index, row in df.iterrows():
    pattern_value = row['pattern']
    patterns.append(pattern_value)

print(len(patterns))
patterns[:1]

# Prompt Definition

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        {instruction}

        ### Input:
        {input}
        ### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction:
        {instruction}
        ### Response:"""



In [ ]:
generation_config = GenerationConfig(
    temperature=1.0,
    top_p=1,
    top_k=40,
    num_beams=4
)

# Description Generation

In [ ]:
instruction = "From the input sentence containing the trigger and the action components of a trigger-action rule, execute the following instruction:" \
              "- Generate a textual description explaining the behaviour of the trigger-action rule"

output_lst = []

for pattern in patterns:
    input = pattern
    print(input)

    prompt = generate_prompt(instruction=instruction, input=input)
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate
    input_ids = inputs["input_ids"].cuda()

    generation_output = alpaca_model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        output_scores=True,
        max_new_tokens=50
    )

    output = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0].split("### Response:")[1].strip()
    output_lst.append(output)